In [50]:
import multiprocessing as mp
from joblib import Parallel, delayed

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#from power import power
#from hyppo import power
#from hyppo-papers import power
from hyppo.independence import CCA, Dcorr, HHG, Hsic, RV, MGC
#from hyppo.sims import *
from hyppo.sims import indep_sim

import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

#from hyppo.random_forest.base import RandomForestTest
#from hyppo.random_forest._utils import _CheckInputs, sim_matrix
from hyppo._utils import perm_test, euclidean
#from hyppo.sims import *
from hyppo.ksample._utils import k_sample_transform
#from hyppo import *
from math import ceil


In [16]:
import seaborn as sns
sns.set(color_codes=True, style='white', context='talk', font_scale=2)
PALETTE = sns.color_palette("Set1")
sns.set_palette(PALETTE[3:])

In [26]:
MAX_SAMPLE_SIZE = 100
STEP_SIZE = 5
SAMP_SIZES = range(5, MAX_SAMPLE_SIZE + STEP_SIZE, STEP_SIZE)
POWER_REPS = 5


SIMULATIONS = {
    "linear": "Linear",
    "multimodal_independence": "Independence"
}

TESTS = [
    RV,
]

In [41]:
def _indep_sim_gen(sim, n, p, noise=True):
    """
    Generate x, y from each sim
    """
    if sim in ["multiplicative_noise", "multimodal_independence"]:
        x, y = indep_sim(sim, n, p)
    else:
        x, y = indep_sim(sim, n, p, noise=noise)

    return x, y

In [42]:
def _perm_stat(test, sim, n=100, p=1, noise=True):
    """
    Generates null and alternate distributions
    """
    x, y = _indep_sim_gen(sim, n, p, noise=noise)
    obs_stat = test()._statistic(x, y)
    permy = np.random.permutation(y)
    perm_stat = test()._statistic(x, permy)

    return obs_stat, perm_stat

In [43]:
def power(test, sim, n=100, p=1, noise=True, alpha=0.05, reps=1000, auto=False):
    """
    Calculates empirical power
    """
    alt_dist, null_dist = map(
        np.float64,
        zip(*[_perm_stat(test, sim, n, p, noise=noise) for _ in range(reps)]),
    )
    cutoff = np.sort(null_dist)[ceil(reps * (1 - alpha))]
    empirical_power = (alt_dist >= cutoff).sum() / reps

    if empirical_power == 0:
        empirical_power = 1 / reps

    return empirical_power

In [44]:
def estimate_power(sim, test):
    est_power = np.array([np.mean([power(test, sim, n=i, p=3) for _ in range(POWER_REPS)])
                          for i in SAMP_SIZES])
    #np.savetxt('../rf/vs_samplesize/{}_{}.csv'.format(sim, test.__name__),
               #est_power, delimiter=',')
    
    #est_power = np.array([np.mean([power_2samp_dimension(test, trans_2samp, sim, p=dim) for _ in range(POWER_REPS)])
                          #for dim in dim_range])
    np.savetxt('C:/Users/siptest/Desktop/NDD/{}_{}.csv'.format(sim.__name__, test.__name__),
                est_power, delimiter=',')
    
    return est_power

In [51]:
outputs = Parallel(n_jobs=-1, verbose=100)(
    [delayed(estimate_power)(sim, test) for sim in SIMULATIONS for test in TESTS]
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    3.2s remaining:    0.0s


TypeError: 'module' object is not callable